In [1]:
import numpy as np
import pandas as pd
import datetime
import re
from dateutil.relativedelta import relativedelta

In [2]:
# 0. 데이터 가져오기

data = pd.read_csv('.\\Weather\\RKSI_TAF.csv', sep='\n', header = None)

data_2018 = pd.read_csv('.\\Weather\\RKSI_TAF_2018.csv', sep='\n', header = None)

TAF_data = list(np.zeros(len(data)))
TAF_data_2018 = list(np.zeros(len(data_2018)))

In [ ]:
##############################################################

# 이부분을 바꿔서 2018도 함

In [3]:
for l in range(0,len(data_2018)):     ##############################################################
    Data = data_2018.iloc[l,0]        ##############################################################
    
    # 1. Pattern
    pattern_issue = '[0-9].....Z'
    pattern_date = '[0-9]+/[0-9]{2,4}'
    pattern_taf = 'TX........Z|TX.......Z'             # BECMG, TEMPO 전 text 찾는데 사용
    pattern_wind = '\s[0-9]...[0-9][A-Z]'              # 단, 100KT이상은 잡을 수 X        
    pattern_vis = '\s[0-9]..[0-9]\s'               
    pattern_cavok = '\sC[A-Z][A-Z][A-Z][A-Z]\s'
    pattern_wc = 'NSW|TS|DZ|RA|SN|SG|IC|PL|GR|GS|BR|FG|FU|VA|DU|SA|HZ|SQ|FC|SS|DS'
    pattern_cloud = '[FSBO][A-Z][A-Z]..[0-9]'
    pattern_chg = 'BECMG|TEMPO'
    regex = re.compile(pattern_date)
    search = regex.search(Data)
    # search.group()
    # search.start()
    # search.end()




    # 2. 발생시간
    yymm = Data[re.search('[0-9]...+-[0-9].+-[0-9].', Data).start() : re.search('[0-9]...+-[0-9].+-[0-9].', Data).start()+8] 
    ddHHMM = Data[re.search(pattern_issue, Data).start() : re.search(pattern_issue, Data).start()+6]
    nextyear = Data[re.search('[0-9]...+-[0-9].+-[0-9].', Data).start()+5 : re.search('[0-9]...+-[0-9].+-[0-9].', Data).start()+7] + Data[re.search(pattern_issue, Data).start() : re.search(pattern_issue, Data).start()+2]

    start = Data[re.search(pattern_date, Data).start() : re.search(pattern_date, Data).start()+4]
    end = Data[re.search(pattern_date, Data).start()+5 : re.search(pattern_date, Data).start()+9]
    start_time = datetime.datetime.strptime(yymm + start ,'%Y-%m-%d%H') + datetime.timedelta(hours = 9)     # india time으로 바꿈!
    if (yymm+end)[-2:] == '24':
        end_time = pd.to_datetime((yymm+end)[:-2], format = '%Y-%m-%d') + pd.Timedelta(days=1) + pd.Timedelta(hours=9)
        end_time = end_time.to_pydatetime()
    else:
        end_time = datetime.datetime.strptime(yymm + end ,'%Y-%m-%d%H') + datetime.timedelta(hours = 9)
        
    if end_time > start_time:
        end_time = end_time
    else: 
        end_time = end_time + relativedelta(months=1)
        start_time = start_time
        
    if nextyear =='1231':
        start_time = start_time + relativedelta(months=1)
        end_time = end_time + relativedelta(months=1)
    else:
        start_time = start_time
        end_time = end_time
            
    column = ['WDIR', 'WSPD', 'WG', 'VIS', 'WC', 'CLA_1LYR', 'BASE_1LYR', 'CLA_2LYR', 'BASE_2LYR', 'CLA_3LYR', 'BASE_3LYR']
    Time = pd.date_range(start = start_time, end = end_time, freq = 'H')
    TAF = pd.DataFrame([], index = Time)





    # 3. BECMG, TEMPO 들어간거 찾기
    chg_iter = re.finditer(pattern_chg, Data)                
    chg = list(chg_iter)





    # 4. 첫번쨰 맨 앞 예보 꺼내고 삭제하기

    # BECMG, TEMPO 앞만 남기고 자르기
    if len(chg) != 0:
        Data = Data[:chg[-len(chg)].end()-6]              

    # 시작시간, 종료시간
    start = Data[re.search(pattern_date, Data).start() : re.search(pattern_date, Data).start()+4]
    end = Data[re.search(pattern_date, Data).start()+5 : re.search(pattern_date, Data).start()+9]
    start_time = datetime.datetime.strptime(yymm + start ,'%Y-%m-%d%H') + datetime.timedelta(hours = 9)     # india time으로 바꿈! 
    if (yymm+end)[-2:] == '24':
        end_time = pd.to_datetime((yymm+end)[:-2], format = '%Y-%m-%d') + pd.Timedelta(days=1) + pd.Timedelta(hours=9)
        end_time = end_time.to_pydatetime()
    else:
        end_time = datetime.datetime.strptime(yymm + end ,'%Y-%m-%d%H') + datetime.timedelta(hours = 9)
    
    if end_time > start_time:
        end_time = end_time
    else: 
        end_time = end_time + relativedelta(months=1)
        start_time = start_time
    if nextyear =='1231':
        start_time = start_time + relativedelta(months=1)
        end_time = end_time + relativedelta(months=1)
    else:
        start_time = start_time
        end_time = end_time
        
    # 바람
    wind_f = re.search(pattern_wind, Data)
    if wind_f != None:
        if wind_f.group()[-1] == 'K':
            wind = Data[re.search(pattern_wind, Data).start()+1 : re.search(pattern_wind, Data).start()+8]
            WDIR = wind[0:3]
            WSPD = wind[3:5]
            WG = '0'
        else:
            wind = Data[re.search(pattern_wind, Data).start()+1 : re.search(pattern_wind, Data).start()+11]
            WDIR = wind[0:3]
            WSPD = wind[3:5]
            WG = wind[6:8]
    else:
        WDIR = '0'
        WSPD = '0'
        WG = '0'

    # 시정   
    if len(re.findall(pattern_cavok, Data)) == 0:
        vis = re.search(pattern_vis, Data)
        if vis != None:
            VIS = Data[re.search(pattern_vis, Data).start()+1 : re.search(pattern_vis, Data).start()+5]
        else:
            VIS = None
    else:
        vis = re.search(pattern_cavok, Data)
        VIS = '9999'    

    # 현천
    wc = re.search(pattern_wc, Data)
    if wc != None:
        if wc.group() == 'NSW':
            WC = '0'
        else:
            WC = wc.group()
    else:
        WC = '0'

    # 구름
    taf_find = re.finditer(pattern_taf, Data)        
    taf_data = Data[:list(taf_find)[-1].start()]
    cloud = re.finditer(pattern_cloud, taf_data) 
    tcloud = list(cloud)
    if len(tcloud) == 3:
        CLA_1LYR = Data[tcloud[0].start() : tcloud[0].start()+3]
        BASE_1LYR = Data[tcloud[0].start()+3 : tcloud[0].start()+6]
        CLA_2LYR = Data[tcloud[1].start() : tcloud[1].start()+3]
        BASE_2LYR = Data[tcloud[1].start()+3 : tcloud[1].start()+6]
        CLA_3LYR = Data[tcloud[2].start() : tcloud[2].start()+3]
        BASE_3LYR = Data[tcloud[2].start()+3 : tcloud[2].start()+6]
    elif len(tcloud) == 2:
        CLA_1LYR = Data[tcloud[0].start() : tcloud[0].start()+3]
        BASE_1LYR = Data[tcloud[0].start()+3 : tcloud[0].start()+6]
        CLA_2LYR = Data[tcloud[1].start() : tcloud[1].start()+3]
        BASE_2LYR = Data[tcloud[1].start()+3 : tcloud[1].start()+6]
        CLA_3LYR = '0'
        BASE_3LYR = '400'
    elif len(tcloud) == 1:
        CLA_1LYR = Data[tcloud[0].start() : tcloud[0].start()+3]
        BASE_1LYR = Data[tcloud[0].start()+3 : tcloud[0].start()+6]
        CLA_2LYR = '0'
        BASE_2LYR = '400'
        CLA_3LYR = '0'
        BASE_3LYR = '400'
    elif len(tcloud) == 0:
        CLA_1LYR = '0'
        BASE_1LYR = '400'
        CLA_2LYR = '0'
        BASE_2LYR = '400'
        CLA_3LYR = '0'
        BASE_3LYR = '400'
    else: 
        CLA_1LYR = Data[tcloud[0].start() : tcloud[0].start()+3]
        BASE_1LYR = Data[tcloud[0].start()+3 : tcloud[0].start()+6]
        CLA_2LYR = Data[tcloud[1].start() : tcloud[1].start()+3]
        BASE_2LYR = Data[tcloud[1].start()+3 : tcloud[1].start()+6]
        CLA_3LYR = Data[tcloud[2].start() : tcloud[2].start()+3]
        BASE_3LYR = Data[tcloud[2].start()+3 : tcloud[2].start()+6]

    weather = [WDIR, WSPD, WG, VIS, WC, CLA_1LYR, BASE_1LYR, CLA_2LYR, BASE_2LYR, CLA_3LYR, BASE_3LYR]

    Data = data_2018.iloc[l,0]         ##############################################################





    # 5. TAF 안에 넣기

    temp_time= pd.date_range(start = start_time, end = end_time, freq = 'H')
    Weather = [weather]

    for i in range(len(temp_time)-1):
        Weather.append(weather)

    Wx = pd.DataFrame(Weather, index = temp_time, columns = column)
    TAF = TAF.combine_first(Wx)
    
    # 5* issue date 붙이기
    issue_date = pd.DataFrame({'issue_time': [str(start_time)]*len(TAF)}, index = TAF.index)
    TAF = issue_date.join(TAF)





    # 6. BECMG, TEMPO 부분 뒤에서 부터 데이터 꺼내오기

    iter = len(chg)

    weathers = list(np.zeros(len(chg)))
    durs = list(np.zeros(len(chg)))

    for i in range(1, iter+1):

        if len(chg) == 0:
            temp = ''
            continue  

        # 마지막 BECMG, TEMPO 뒤만 남기고 자르기
        temp = Data[chg[-1].end()-6:]
        
        # 시작시간, 종료시간
        temp_start = temp[re.search(pattern_date, temp).start() : re.search(pattern_date, temp).start()+4]   
        temp_end = temp[re.search(pattern_date, temp).start()+5 : re.search(pattern_date, temp).start()+9]
        temp_start_time = datetime.datetime.strptime(yymm + temp_start ,'%Y-%m-%d%H') + datetime.timedelta(hours = 9)     
        if (yymm+temp_end)[-2:] == '24':
            temp_end_time = pd.to_datetime((yymm+temp_end)[:-2], format = '%Y-%m-%d') + pd.Timedelta(days=1) + pd.Timedelta(hours=9)
            temp_end_time = temp_end_time.to_pydatetime()
        else:
            temp_end_time = datetime.datetime.strptime(yymm + temp_end ,'%Y-%m-%d%H') + datetime.timedelta(hours = 9)         
        
        if temp_end_time > temp_start_time:
            temp_end_time = temp_end_time
        else: 
            temp_end_time = temp_end_time + relativedelta(months=1)
            temp_start_time = temp_start_time
        
        if nextyear =='1231':
            temp_start_time = temp_start_time + relativedelta(months=1)
            temp_end_time = temp_end_time + relativedelta(months=1)
        else:
            temp_start_time = temp_start_time
            temp_end_time = temp_end_time
                    
        # 바람    
        temp_wind_f = re.search(pattern_wind, temp)
        if temp_wind_f != None:
            if temp_wind_f.group()[-1] == 'K':
                temp_wind = temp[re.search(pattern_wind, temp).start()+1 : re.search(pattern_wind, temp).start()+8]
                WDIR = temp_wind[0:3]
                WSPD = temp_wind[3:5]
                WG = '0'
            else:
                temp_wind = temp[re.search(pattern_wind, temp).start()+1 : re.search(pattern_wind, temp).start()+11]
                WDIR = temp_wind[0:3]
                WSPD = temp_wind[3:5]
                WG = temp_wind[6:8]
        else:
            WDIR = None
            WSPD = None
            WG = None                
        # 현천
        temp_wc = re.search(pattern_wc, temp)
        if temp_wc != None:
            if temp_wc.group() == 'NSW':
                WC = '0'
            else:
                WC = temp_wc.group()
        else:
            WC = '0'
        # 구름    
        temp_cloud = re.finditer(pattern_cloud, temp) 
        temp_tcloud = list(temp_cloud)
        if len(temp_tcloud) == 3:
            CLA_1LYR = temp[temp_tcloud[0].start() : temp_tcloud[0].start()+3]
            BASE_1LYR = temp[temp_tcloud[0].start()+3 : temp_tcloud[0].start()+6]
            CLA_2LYR = temp[temp_tcloud[1].start() : temp_tcloud[1].start()+3]
            BASE_2LYR = temp[temp_tcloud[1].start()+3 : temp_tcloud[1].start()+6]
            CLA_3LYR = temp[temp_tcloud[2].start() : temp_tcloud[2].start()+3]
            BASE_3LYR = temp[temp_tcloud[2].start()+3 : temp_tcloud[2].start()+6]
        elif len(temp_tcloud) == 2:
            CLA_1LYR = temp[temp_tcloud[0].start() : temp_tcloud[0].start()+3]
            BASE_1LYR = temp[temp_tcloud[0].start()+3 : temp_tcloud[0].start()+6]
            CLA_2LYR = temp[temp_tcloud[1].start() : temp_tcloud[1].start()+3]
            BASE_2LYR = temp[temp_tcloud[1].start()+3 : temp_tcloud[1].start()+6]
            CLA_3LYR = '0'
            BASE_3LYR = '400'
        elif len(temp_tcloud) == 1:
            CLA_1LYR = temp[temp_tcloud[0].start() : temp_tcloud[0].start()+3]
            BASE_1LYR = temp[temp_tcloud[0].start()+3 : temp_tcloud[0].start()+6]
            CLA_2LYR = '0'
            BASE_2LYR = '400'
            CLA_3LYR = '0'
            BASE_3LYR = '400'
        else: 
            CLA_1LYR = None
            BASE_1LYR = None
            CLA_2LYR = None
            BASE_2LYR = None
            CLA_3LYR = None
            BASE_3LYR = None
        # 시정  
        if len(re.findall(pattern_cavok, temp)) == 0:
            temp_vis = re.search(pattern_vis, temp)
            if temp_vis != None:
                VIS = temp[re.search(pattern_vis, temp).start()+1 : re.search(pattern_vis, temp).start()+5]
            else:
                VIS = None
        else:
            temp_vis = re.search(pattern_cavok, temp)
            VIS = '9999'
            CLA_1LYR = '0'
            BASE_1LYR = '0'
            CLA_2LYR = '0'
            BASE_2LYR = '0'
            CLA_3LYR = '0'
            BASE_3LYR = '0'
            

        weather = [WDIR, WSPD, WG, VIS, WC, CLA_1LYR, BASE_1LYR, CLA_2LYR, BASE_2LYR, CLA_3LYR, BASE_3LYR]
        dur = [chg[-1].group(), temp_start_time, temp_end_time]

        Data = Data[:chg[-1].end()-6]
        chg_iter = re.finditer(pattern_chg, Data)                # BECMG, TEMPO 들어간거 찾기
        chg = list(chg_iter)

        weathers[-i] = weather
        durs[-i] = dur





    # 7. 각각의 weather를 맞는 시간대의 TAF에 넣기
    WW = TAF

    for j in range(len(durs)):  
        if durs[j][0] == 'BECMG':
            durs[j][2] = end_time
            temp_time = pd.date_range(start = durs[j][1], end = durs[j][2], freq = 'H')
            Weather = [weathers[j]]
            for k in range(len(temp_time)-1):
                Weather.append(weathers[j])
            Wx = pd.DataFrame(Weather, index = temp_time, columns = column)
            issue_date = pd.DataFrame({'issue_time': [str(start_time)]*len(Wx)}, index = Wx.index)    # issue date 넣기
            Wx = issue_date.join(Wx)
            TAF.loc[str(durs[j][1]) : str(durs[j][2])] = Wx
            WW = TAF.fillna(method = 'ffill')
        else:
            temp_time= pd.date_range(start = durs[j][1], end = durs[j][2], freq = 'H')
            Weather = [weathers[j]]
            for k in range(len(temp_time)-1):
                Weather.append(weathers[j])
            Wx = pd.DataFrame(Weather, index = temp_time, columns = column)
            issue_date = pd.DataFrame({'issue_time': [str(start_time)]*len(Wx)}, index = Wx.index)    # issue date 넣기
            Wx = issue_date.join(Wx)
            TAF.loc[str(durs[j][1]) : str(durs[j][2])] = Wx
            WW = TAF.fillna(method = 'ffill')  
    
    # 8.통합
    TAF_data_2018[l] = WW

In [6]:
# csv로 저장    
# pd.concat(TAF_data).to_csv('C:\\Users\\user\\proj\\TMA_Capacity\\data\\Weather\\TAF_data.csv')

pd.concat(TAF_data_2018).to_csv('C:\\Users\\user\\proj\\TMA_Capacity\\data\\Weather\\TAF_data_2018.csv')

In [ ]:
pd.options.display.max_columns = None

print(data.iloc[1000,0])
TAF_data[1000]

In [4]:
pd.options.display.max_columns = None

print(data_2018.iloc[1000,0])
TAF_data_2018[1000]

taf	RKSI	2018-07-16 20:27:00	TAF \n AMD TAF \n AMD RKSI 162025Z 1620/1724 24005KT 1800 BR FEW010 SCT030 TN23/1621Z TX30/1706Z TN23/1720Z \n TEMPO 1620/1623 0600 FG OVC002 \n BECMG 1623/1624 29006KT 6000 NSW \n BECMG 1718/1719 26005KT 4200 BR \n TEMPO 1719/1722 1200 \n BECMG 1722/1723 7000 NSW


,issue_time,WDIR,WSPD,WG,VIS,WC,CLA_1LYR,BASE_1LYR,CLA_2LYR,BASE_2LYR,CLA_3LYR,BASE_3LYR
2018-07-17 05:00:00,2018-07-17 05:00:00,None,None,None,0600,FG,OVC,002,0,400,0,400
2018-07-17 06:00:00,2018-07-17 05:00:00,None,None,None,0600,FG,OVC,002,0,400,0,400
2018-07-17 07:00:00,2018-07-17 05:00:00,None,None,None,0600,FG,OVC,002,0,400,0,400
2018-07-17 08:00:00,2018-07-17 05:00:00,290,06,0,6000,0,OVC,002,0,400,0,400
2018-07-17 09:00:00,2018-07-17 05:00:00,290,06,0,6000,0,OVC,002,0,400,0,400
2018-07-17 10:00:00,2018-07-17 05:00:00,290,06,0,6000,0,OVC,002,0,400,0,400
2018-07-17 11:00:00,2018-07-17 05:00:00,290,06,0,6000,0,OVC,002,0,400,0,400
2018-07-17 12:00:00,2018-07-17 05:00:00,290,06,0,6000,0,OVC,002,0,400,0,400
2018-07-17 13:00:00,2018-07-17 05:00:00,290,06,0,6000,0,OVC,002,0,400,0,400
2018-07-17 14:00:00,2018-07-17 05:00:00,290,06,0,6000,0,OVC,002,0,400,0,400


In [ ]:
################################################################################
# 12월 31일 넘어가면서 한달씩 늘리는거 다시 고치기
# 구름에 CB있는거 넣을수 있도록




################################################################################